# Predicting if sequence is linear or not
this is a tutorial on how to implement a Dynamcic LSTM.

In [48]:
import numpy as np

In [49]:
import tensorflow as tf

# Creating linear and random sequences
class 0 == linear sequences [10,11,12...]
class 1 == random sequences [10,20,1,...]

In [50]:
print(tf.__version__)

1.12.0


In [51]:
import random
n_samples = 1500
max_seq_len = 10
min_seq_len = 5
max_val = 200
data = []
labels = []

for i in range(n_samples):
    len1=random.randint(min_seq_len,max_seq_len)
    if(random.random() < .5) :
        start = random.randint(0,max_val-len1)
        s = [float(i) for i in range(start,start+len1)]
        s=np.asarray(s)
        data.append(s)
        labels.append([1,0])
    else :
        s = [float(random.randint(0,max_val)) for i in range(len1)]
        s = np.asarray(s)
        data.append(s)
        labels.append([0,1])

In [65]:
data = np.asarray(data)
labels = np.asarray(labels)
print(data[5:10])
print(labels[5:10])

[array([ 42.,   2.,  26., 129., 122., 110.])
 array([43., 44., 45., 46., 47.])
 array([ 32., 102.,   5., 164., 195.,  22.])
 array([90., 91., 92., 93., 94., 95., 96., 97., 98.])
 array([21., 22., 23., 24., 25., 26.])]
[[0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]]


In [59]:
data.shape

(1500,)

# Model declaration

In [70]:
batch_size = 50
display_step = 100

num_hidden = 64
num_classes = 2

num_layers = 2

learning_rate = tf.placeholder(tf.float32,name='lr_rate')
keep_prob = tf.placeholder(tf.float32,name='keep_prob')

seqlen = tf.placeholder(tf.int32,[None],name='sequence_length')

sequence_maximum_length = 20 #tf.reduce_max(seqlen)

x = tf.placeholder("float",[None,None,1])
y = tf.placeholder("float",[None,num_classes])

cell = tf.contrib.rnn.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(rnn_size), keep_prob)

# x = tf.unstack(x,sequence_maximum_length,4)


# stacked_cells = tf.contrib.rnn.MultiRNNCell(
#     [tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(num_hidden), keep_prob) for _ in range(num_layers)])

##outputs obtained here have max_length of batch but computation is done only until
init_state = stacked_cells.zero_state(batch_size, tf.float32)
outputs, last_states = tf.nn.dynamic_rnn(cell,x,initial_state=init_state,sequence_length=seqlen,dtype=tf.float32)


#softmax layer for output

W = tf.Variable(tf.random_normal([num_hidden,num_classes]))

b = tf.Variable(tf.random_normal([num_classes]))


##We need to get outputs at which dynamic rnn computation is stopped

outputs = tf.stack(outputs)

# Outputs of _dynamic_rnn_loop are always shaped [time, batch, depth].
#so converting it to [batch,time,depth]
outputs = tf.transpose(outputs, [1, 0, 2])

index = tf.range(0,batch_size) * sequence_maximum_length + (seqlen -1)

#inputs to softmax layers [batch_size*num_hidden]
outputs = tf.gather(tf.reshape(outputs,[-1,num_hidden]),index)

#logits 
preds = tf.matmul(outputs,W)+b

#Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

#Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

#Initialize the variables
init = tf.gloabal_variables_initializer()

ValueError: Variable rnn/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\SAIRAM\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\ops.py", line 1770, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\SAIRAM\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\ops.py", line 3274, in create_op
    op_def=op_def)
  File "C:\Users\SAIRAM\AppData\Local\conda\conda\envs\tensorflow_env\lib\site-packages\tensorflow\python\util\deprecation.py", line 488, in new_func
    return func(*args, **kwargs)


In [54]:
o = np.arange(30).reshape((2,3,5))
print(o)
xx = tf.unstack(o)
o = tf.stack(o)
# o = tf.transpose(o,[1,0,2])
seqlen = np.array([3,1])
# print(seqlen-1)
seq_max_len = 3
index = seqlen - 1
index = tf.range(0,2)*3 + (seqlen - 1 )
o = tf.reshape(o,[-1,5])
o = tf.gather(tf.reshape(o,[-1,5]),index)
with tf.Session() as sess:
    print(sess.run(index))
    print(sess.run(xx))
#tf.transpose(o,[1,0,2])

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]]

 [[15 16 17 18 19]
  [20 21 22 23 24]
  [25 26 27 28 29]]]
[2 3]
[array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]]), array([[15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29]])]


In [74]:
data_train = data[:1300]
label_train =labels[:1300]
data_test = data[1300:]
label_test = labels[:1300]
# for src in data_train:
#     print(src.shape[0])
print(data_train[3].shape)
print(label_train.shape[0])

(7,)
1300


In [75]:
def get_batches(sources,batch_size):
    """Batch targets, sources, and the lengths of their sentences together"""
    # print(len(sources)//batch_size)
    for batch_i in range(0, sources.shape[0]//batch_size):
        start_i = batch_i * batch_size
        seq_length = []
        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]
        
        for source in sources_batch:
            seq_length.append(source.shape[0])
        
        yield sources_batch,targets_batch,

In [77]:
num_epochs=10
with tf.Session() as sess:
    sess.run(init)
    for step in range(1,training_steps+1):
        for batch_i,(source_batch, labels_batch,source_lengths) in enumerate(get_batches(data_train,batch_size))
            sess.run(optimizer,feed_dict={x:source_batch,y:labels_batch,seqlen:source_lengths})
            if step % displa_step == 0 or step == 1:
                acc,loss = sess.run([accuracy,cost],feed_dict={x:source_batch,y:labels_batch,seqlen:source_lengths})
                print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

SyntaxError: invalid syntax (<ipython-input-77-6852ac213504>, line 5)